<a href="https://colab.research.google.com/github/technologyhamed/Neuralnetwork/blob/Single/DepthwiseSeparableConvolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.datasets import mnist

# load dataset
(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()

In [13]:
class DepthwiseSeparableConv2D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding, activation):
    super(DepthwiseSeparableConv2D, self).__init__()
    self.depthwise =  tf.keras.layers.DepthwiseConv2D(kernel_size = kernel_size, padding = padding, activation = activation)
    self.pointwise =  tf.keras.layers.Conv2D(filters = filters, kernel_size = (1, 1), activation = activation)

  def call(self, input_tensor):
    x = self.depthwise(input_tensor)
    return self.pointwise(x)

In [16]:
visible = keras.Input(shape=(32, 32, 3))
depthwise_separable = DepthwiseSeparableConv2D(filters=64, kernel_size=(5,5), padding="valid", activation="relu")(visible)
depthwise_model = keras.Model(inputs=visible, outputs=depthwise_separable)
depthwise_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 depthwise_separable_conv2d_  (None, 28, 28, 64)       334       
 6 (DepthwiseSeparableConv2D                                     
 )                                                               
                                                                 
Total params: 334
Trainable params: 334
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, SeparableConv2D
import tensorflow as tf

# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = Conv2D(filters = n_filters, kernel_size = (3,3), padding='same', activation="relu")(layer_in)
	# add max pooling layer
	layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in

visible = Input(shape=(32, 32, 3))
layer = vgg_block(visible, 64, 2)
layer = vgg_block(layer, 128, 2)
layer = vgg_block(layer, 256, 2)
layer = Flatten()(layer)
layer = Dense(units=10, activation="softmax")(layer)

# create model
model = Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")

history = model.fit(x=trainX, y=trainY, batch_size=128, epochs=10, validation_data=(testX, testY))

In [24]:
import tensorflow.keras as keras
from keras.datasets import mnist
import tensorflow as tf

# load dataset
(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
# depthwise separable VGG block
def vgg_depthwise_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = tf.keras.layers.SeparableConv2D(filters = n_filters, kernel_size = (3,3), padding='same',activation='relu')(layer_in)
	# add max pooling layer
	layer_in = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in

visible = keras.Input(shape=(32, 32, 3))
layer = vgg_depthwise_block(visible, 64, 2)
layer = vgg_depthwise_block(layer, 128, 2)
layer = vgg_depthwise_block(layer, 256, 2)
layer = tf.keras.layers.Flatten()(layer)
layer = tf.keras.layers.Dense(units=10, activation="softmax")(layer)
# create model
model = tf.keras.Model(inputs=visible, outputs=layer)

# summarize model
model.summary()

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")

history_dsconv = model.fit(x=trainX, y=trainY, batch_size=128, epochs=10, validation_data=(testX, testY))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 separable_conv2d_26 (Separa  (None, 32, 32, 64)       283       
 bleConv2D)                                                      
                                                                 
 separable_conv2d_27 (Separa  (None, 32, 32, 64)       4736      
 bleConv2D)                                                      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 separable_conv2d_28 (Separa  (None, 16, 16, 128)      8896      
 bleConv2D)                                                

KeyboardInterrupt: ignored